In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np
import pandas as pd
from prepare_regression_data import save_aggregated_indiv_day_data

In [4]:
OMICRON_START_DATE = '2021-12-05'
OMICRON_END_DATE = '2021-12-31'

## Preparing data for regression (each csv takes ~12min)

In [ ]:
delays = list(range(1, 15))

for i in delays:
    print(i)
    save_aggregated_indiv_day_data(booster_delay=i)

## Covariate distribution (7-day delay)

In [6]:
df_7 = pd.read_csv(f"../data/regression/data_indiv_day_delay_7.csv")

df_by_booster = df_7.groupby(['booster', 'infection'], as_index=False)['employee_id_hash'].count()
df_count = pd.pivot_table(df_by_booster, values='employee_id_hash', index=['booster'], columns=['infection'], aggfunc=np.sum, margins=True)
print(df_count)

sorter_sg =['UG-greek', 'UG-athlete', 'UG-other', 'GR', 'LA', 'GM', 'VM']
df_7.student_group = df_7.student_group.astype("category")
df_7.student_group.cat.set_categories(sorter_sg, inplace=True)
sorter_vac = ['Pfizer', 'Moderna', 'J&J', 'Others']
df_7.vac_type_agg = df_7.vac_type_agg.astype("category")
df_7.vac_type_agg.cat.set_categories(sorter_vac, inplace=True)

for feature in ['student_group', 'last_dose_month', 'current_gender', 'vac_type_agg', 'week']:
    df_agg = df_7[df_7['booster'] == 0].groupby([feature, 'infection'], as_index=False)['employee_id_hash'].count()                     
    df_agg_count = pd.pivot_table(df_agg, values='employee_id_hash', index=[feature], columns=['infection'], aggfunc=np.sum, margins=True)
    print('unboosted\n', df_agg_count, '\n')

    df_agg = df_7[df_7['booster'] == 1].groupby([feature, 'infection'], as_index=False)['employee_id_hash'].count()                     
    df_agg_count = pd.pivot_table(df_agg, values='employee_id_hash', index=[feature], columns=['infection'], aggfunc=np.sum, margins=True)
    print('boosted\n', df_agg_count, '\n')

C:\Users\jw2529\Anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\jw2529\Anaconda3\envs\py37\lib\site-packages\pandas\core\arrays\categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


infection       0     1     All
booster                        
0          133415  1799  135214
1           19715   127   19842
All        153130  1926  155056
unboosted
 infection           0     1     All
student_group                      
UG-greek        20816   517   21333
UG-athlete       7459   170    7629
UG-other        88875   985   89860
GR                  0     0       0
LA               6136    30    6166
GM               4895    84    4979
VM               5234    13    5247
All            133415  1799  135214 

boosted
 infection          0    1    All
student_group                   
UG-greek        2951   35   2986
UG-athlete       774    7    781
UG-other       11381   65  11446
GR                 0    0      0
LA              1731    3   1734
GM              1046   15   1061
VM              1832    2   1834
All            19715  127  19842 

unboosted
 infection               0       1     All
last_dose_month                          
1                   273.0     5